**Building an Emotion Classifier Model**

In [81]:
#Lets import the dataset
import pandas as pd
data = pd.read_csv('/content/ISEAR_dataset (1).csv')
data.head()

,joy,On days when I feel close to my partner and other friends. \nWhen I feel at peace with myself and also experience a close \ncontact with people whom I regard greatly.,Unnamed: 2
0,fear,Every time I imagine that someone I love or I ...,NaN
1,anger,When I had been obviously unjustly treated and...,NaN
2,sadness,When I think about the short time that we live...,NaN
3,disgust,At a gathering I found myself involuntarily si...,NaN
4,shame,When I realized that I was directing the feeli...,NaN


In [82]:
data.columns = ['EMOTION', 'TEXT', 'Unnamed']
data = data.drop(columns=['Unnamed'])
data.head()

,EMOTION,TEXT
0,fear,Every time I imagine that someone I love or I ...
1,anger,When I had been obviously unjustly treated and...
2,sadness,When I think about the short time that we live...
3,disgust,At a gathering I found myself involuntarily si...
4,shame,When I realized that I was directing the feeli...


In [83]:
#Importing the libraries for building Emotion Classifier

from nltk.corpus import stopwords

from nltk.stem.wordnet import WordNetLemmatizer

import string

from textblob.classifiers import NaiveBayesClassifier

from textblob import TextBlob

from nltk.corpus import stopwords

from nltk.stem import PorterStemmer

from textblob import Word

from nltk.util import ngrams

import re

from nltk.tokenize import word_tokenize

import matplotlib.pyplot as plt

from sklearn.feature_extraction. text import CountVectorizer, TfidfVectorizer

from sklearn.decomposition import LatentDirichletAllocation

import sklearn.feature_extraction.text as text

from sklearn.decomposition import NMF, LatentDirichletAllocation,TruncatedSVD

from sklearn import model_selection, preprocessing, linear_model,naive_bayes, metrics, svm

import xgboost

from sklearn import decomposition, ensemble

import pandas, numpy, textblob, string

import re

import nltk

from sklearn.metrics import classification_report

from sklearn.metrics import confusion_matrix

from sklearn.metrics import accuracy_score

from sklearn.metrics import mean_absolute_error

In [84]:
#Convert uppercase letters to lowercase.
data['TEXT'] = data['TEXT'].apply(lambda a: " ".join(a.lower() for a in a.split()))

In [85]:
#Remove white space and special characters.
data['TEXT'] = data['TEXT'].apply(lambda a: " ".join(a.replace('[^\w\s]','') for a in a.split()))

In [86]:
#Remove the stop words.
stop = stopwords.words('english')
data['TEXT'] = data['TEXT'].apply(lambda a: " ".join(a for a in a.split() if a not in stop))

In [87]:
#Correct Spelling
data['TEXT'] = data['TEXT'].apply(lambda a: str(TextBlob(a).correct()))

In [88]:
#Do Stemming
st = PorterStemmer()
data['TEXT'] = data['TEXT'].apply(lambda a: " ".join([st.stem (word) for word in a.split()]))

In [89]:
#After completing all the preprocessing steps, this is what the data looks like.
data.head()

,EMOTION,TEXT
0,fear,everi time imagin someon love could contact se...
1,anger,obvious unjustli treat possibl elucid this.
2,sadness,think short time live relat period life think ...
3,disgust,gather found involuntarili sit next two peopl ...
4,shame,realiz direct feel discont partner way tri put...


In [90]:
#data before encoding.
data['EMOTION'].value_counts()

joy        1091
sadness    1082
anger      1079
fear       1076
shame      1071
disgust    1066
guilt      1049
guit          1
Name: EMOTION, dtype: int64

In [91]:
#Labels encode the target variable
object = preprocessing. LabelEncoder()
data['EMOTION'] = object. fit_transform(data['EMOTION'])

In [92]:
# data after encoding.
data['EMOTION'].value_counts()

5    1091
6    1082
0    1079
2    1076
7    1071
1    1066
3    1049
4       1
Name: EMOTION, dtype: int64

In [93]:
from sklearn.model_selection import train_test_split

# Get the emotions with at least two instances
valid_emotions = data['EMOTION'].value_counts()[data['EMOTION'].value_counts() > 1].index.tolist()

# Filter the data to only include valid emotions
filtered_data = data[data['EMOTION'].isin(valid_emotions)]

# Split the filtered data into train and test sets
Xtrain, Xtest, Ytrain, Ytest = train_test_split(filtered_data['TEXT'], filtered_data['EMOTION'], stratify=filtered_data['EMOTION'])

**Feature Engineering**

In [94]:
cv = CountVectorizer()

cv.fit(data['TEXT'])

cv_xtrain = cv.transform(Xtrain)

cv_xtest = cv. transform(Xtest)

# word-level TF-IDF

tv = TfidfVectorizer()

tv.fit(data['TEXT'])
#Transform the training and validation data using TF-IDF object.

tv_xtrain = tv.transform(Xtrain)

tv_xtest =tv.transform(Xtest)

**Model Building**

In [95]:
def build(model_initializer, independent_variables_training, target, independent_variable_test):
    model_initializer.fit(independent_variables_training, target)
    modelPred = model_initializer.predict(independent_variable_test)
    return metrics.accuracy_score(modelPred, Ytest)

**Multinominal Naive Bayes**

In [96]:
# The following uses naive Bayes generated with count vectors.
output = build(naive_bayes.MultinomialNB(), cv_xtrain, Ytrain, cv_xtest)
print(output)
# The following uses naive Bayes generated with word-level TF-IDF vectors.
output = build (naive_bayes. MultinomialNB (), tv_xtrain, Ytrain, tv_xtest)
print(output)

0.5689196381053752
0.572645023948909


**Linear Classifier/Logistic Regression**

In [97]:
from sklearn.linear_model import LogisticRegression

# Set a higher max_iter value
max_iter_value = 1000  # You can adjust this value based on your data

# for CV
output_cv = build(LogisticRegression(max_iter=max_iter_value), cv_xtrain, Ytrain, cv_xtest)
print(output_cv)

# for TF-IDF
output_tfidf = build(LogisticRegression(max_iter=max_iter_value), tv_xtrain, Ytrain, tv_xtest)
print(output_tfidf)

0.5731772219265566
0.5965939329430548


**Support Vector Machine**

In [98]:
#for cv

output =build(svm. SVC(), cv_xtrain, Ytrain, cv_xtest)
print (output)

#for TF-IDF

output =build (svm.SVC(), tv_xtrain, Ytrain, tv_xtest)
print(output)

0.5604044704630122
0.5864821713677488


**Random Forest**

In [99]:
#for CV

output = build(ensemble. RandomForestClassifier(), cv_xtrain, Ytrain, cv_xtest)
print(output)

#for TF-IDF

output = build(ensemble. RandomForestClassifier(), tv_xtrain, Ytrain,tv_xtest)

print(output)

0.5518893028206493
0.5572112825971262
